In [1]:
import pandas as pd
import requests
import googlemaps
from datetime import datetime

In [2]:
df = pd.read_csv("../raw_data/traffic/traffic.csv")

df = df[df["Date"].str.contains("2020") == True]
df['traffic'] = (df.iloc[:, 5:].sum(axis=1)) / 24
df = df[['ID', 'Roadway Name', 'To', 'traffic']].copy()

/tmp/ipykernel_58377/3097917546.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['traffic'] = (df.iloc[:, 5:].sum(axis=1)) / 24


In [3]:
df_mean = df.groupby(['ID']).mean()
df_mean = df_mean.reset_index()

In [4]:
df = df.drop(['traffic'], axis=1)
df = df.drop_duplicates()

In [5]:
traffic = df.set_index('ID').join(df_mean.set_index('ID'))
traffic = traffic.reset_index()
traffic = traffic.drop_duplicates(subset=['ID'], keep='first')

In [8]:
gmaps = googlemaps.Client(key='GOOGLE_API_KEY')

In [9]:
lat_long = []
for i in range(len(traffic)):
    first = traffic.iloc[i][1]
    second = traffic.iloc[i][2]
    geocode_result = gmaps.geocode(f'{first}, {second}, NY')
    coords = (geocode_result[0]['geometry']['location']['lat'], geocode_result[0]['geometry']['location']['lng'])
    lat_long.append(coords)

In [10]:
lat = []
lng = []
for i in range(349):
    lat.append(lat_long[i][0])
    lng.append(lat_long[i][1])

In [11]:
traffic = traffic.drop(['Roadway Name', 'To', 'ID'], axis=1)
traffic['latitude'] = lat
traffic['longitude'] = lng

In [12]:
traffic

,traffic,latitude,longitude
0,954.828704,40.760866,-73.961061
1,1027.875000,40.800984,-73.931779
2,662.504630,40.730013,-73.983522
3,684.055556,40.739396,-73.976695
4,976.597222,40.762194,-73.993613
...,...,...,...
368,337.578704,40.759920,-73.958829
370,303.451389,40.768816,-73.952325
371,752.300926,40.747521,-73.993776
372,165.064815,40.821958,-73.928103


In [14]:
traffic.to_csv('../processed_data/geocoded_traffic.csv', index=False, encoding='utf-8')